In [ ]:
# First, install the necessary libraries
!pip install langchain langchain_openai

import os
from datetime import datetime
from langchain_openai import ChatOpenAI

# Set up your OpenAI API key
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except ImportError:
    print("Not in a Colab environment, assuming API key is set.")

In [ ]:
# --- 1. The Tool ---
# An agent's power comes from its tools. Let's define a very simple one.
# This is just a standard Python function.
def get_current_time(*args, **kwargs):
    """
    A tool that returns the current date and time as a string.
    It takes no arguments.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
# --- 2. The Agent's "Brain" ---
# This is the LLM that will do the reasoning.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
# --- 3. The ReAct Prompt ---
# This is the most critical piece. We are teaching the LLM how to behave.
# We give it instructions and, most importantly, describe the tools it has access to.
prompt_template = """
You are an assistant that has access to a tool for finding the current time.
You must answer the user's question.

To use a tool, you can use the following format:
Thought: I need to use a tool to find the information.
Action: [tool_name]

The result of the tool will be returned to you in the following format:
Observation: [tool_output]

If you have enough information to answer the user's question, you must respond with the final answer in the following format:
Thought: I have enough information to answer the user's question.
Final Answer: [your final answer]

Here are the tools you have access to:
- get_current_time: A tool that returns the current date and time as a string. It takes no arguments.

Let's begin.

User Question: What is the current time in words?
{scratchpad}
"""

In [ ]:
# --- 4. The Manual ReAct Loop ---
# We will manually orchestrate the Thought -> Action -> Observation loop.

# We'll use a "scratchpad" to keep track of the agent's thoughts and observations
scratchpad = ""
max_loops = 5
loop_count = 0

while loop_count < max_loops:
    loop_count += 1
    print(f"--- Loop {loop_count} ---")

    # Construct the full prompt for this loop
    full_prompt = prompt_template.format(scratchpad=scratchpad)

    # Get the LLM's response
    llm_response = llm.invoke(full_prompt).content
    print(f"LLM Response:\n{llm_response}")

    # Check if the LLM provided a final answer
    if "Final Answer:" in llm_response:
        final_answer = llm_response.split("Final Answer:")[-1].strip()
        print(f"\n--- Final Answer Found ---")
        print(final_answer)
        break

    # If not, check if it wants to use a tool
    if "Action:" in llm_response:
        # Parse out the action
        action_str = llm_response.split("Action:")[-1].strip()

        # For this simple example, we know the only tool is "get_current_time"
        if action_str == "get_current_time":
            tool_output = get_current_time()
            print(f"Tool Output: {tool_output}")

            # Update the scratchpad with the new thought, action, and observation
            scratchpad += f"\n{llm_response}\nObservation: {tool_output}"
        else:
            scratchpad += f"\nInvalid tool '{action_str}' requested."
            print(f"Invalid tool '{action_str}' requested.")

    else:
        print("LLM did not provide a valid action or final answer.")
        break